In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleaned-ecommerce-data/2019-Dec-Cleaned.csv
/kaggle/input/cleaned-ecommerce-data/2019-Nov-Cleaned.csv


In [5]:
# File path for the November dataset
november_file = "/kaggle/input/cleaned-ecommerce-data/2019-Nov-Cleaned.csv"

# Load the November dataset
november_df = pd.read_csv(november_file)

# Display November data preview
print("November Data Preview:")
print(november_df.head())

November Data Preview:
                event_time event_type  product_id          category_id  \
0  2019-11-01 00:00:00 UTC       view     1003461  2053013555631882655   
1  2019-11-01 00:00:00 UTC       view     5000088  2053013566100866035   
2  2019-11-01 00:00:01 UTC       view     3601530  2053013563810775923   
3  2019-11-01 00:00:01 UTC       view     1004775  2053013555631882655   
4  2019-11-01 00:00:01 UTC       view     1306894  2053013558920217191   

               category_code   brand   price    user_id  \
0     electronics.smartphone  xiaomi  489.07  520088904   
1  appliances.sewing_machine  janome  293.65  530496790   
2  appliances.kitchen.washer      lg  712.87  518085591   
3     electronics.smartphone  xiaomi  183.27  558856683   
4         computers.notebook      hp  360.09  520772685   

                           user_session  
0  4d3b30da-a5e4-49df-b1a8-ba5943f1dd33  
1  8e5f4f83-366c-4f70-860e-ca7417414283  
2  3bfb58cd-7892-48cc-8020-2f17e6de6e7f  
3  313628

In [6]:
# Feature 1: Total Spent
november_df["total_spent"] = november_df["price"]

# Feature 2: Extract Date-Time Features
november_df["event_time"] = pd.to_datetime(november_df["event_time"])
november_df["hour"] = november_df["event_time"].dt.hour
november_df["day"] = november_df["event_time"].dt.day
november_df["month"] = november_df["event_time"].dt.month

# Feature 3: Aggregate User Data
user_agg_nov = november_df.groupby("user_id").agg(
    avg_spent_per_user=("total_spent", "mean"),
    num_sessions=("user_session", "nunique")  # Count unique sessions per user
).reset_index()

# Save the processed features
user_agg_nov.to_csv("/kaggle/working/november_processed_data.csv", index=False)

In [7]:
# XGBoost Section
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Add dummy 'target' column for demonstration (replace with actual target column if available)
import numpy as np
user_agg_nov["target"] = np.random.choice([0, 1], size=len(user_agg_nov))

# Split features (X) and target (y)
X = user_agg_nov.drop(["user_id", "target"], axis=1)
y = user_agg_nov["target"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost model
model = XGBClassifier()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.4996707923861352
